In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

# URL 설정
url = 'https://finance.naver.com/sise/sise_market_sum.nhn?&page='

# 데이터 저장 리스트 및 카운터
company_data = []
count = 0

# 페이지 별로 데이터 수집
for page in range(1, 40):
    res = requests.get(url + str(page))
    html = res.content.decode('euc-kr', 'replace')
    soup = BeautifulSoup(html, 'lxml')
    stock_table = soup.find('table', attrs={'class': 'type_2'})
    stock_rows = stock_table.find_all('tr', onmouseover=True)

    # 주식 데이터 파싱
    for row in stock_rows:
        if count >= 30:  # 기업 수를 10개로 제한
            break

        stock_data = row.find_all('td')
        
        if len(stock_data) > 1:
            per = stock_data[11].text.strip()
            if per != 'N/A':
                company_name = stock_data[1].text.strip()
                company_code = stock_data[1].find('a')['href'].split('code=')[-1]
                company_code = "'" + company_code.zfill(6)

                company_data.append([company_name, company_code])
                count += 1
    if count >= 10:  # 10개 수집 시 루프 종료
        break

# DataFrame으로 변환
df = pd.DataFrame(data=company_data, columns=['Company', 'Code'])

# Data 폴더 생성 (폴더가 없으면 생성)
data_folder = 'Data'
os.makedirs(data_folder, exist_ok=True)

# CSV 파일 저장
output_path = os.path.join(data_folder, 'kopsi_stocks.csv')
df.to_csv(output_path, index=False, encoding='utf-8-sig')

# 결과 출력
print(f"파일이 {output_path}에 저장되었습니다.")
print(df)


파일이 Data\kopsi_stocks.csv에 저장되었습니다.
      Company     Code
0        삼성전자  '005930
1      SK하이닉스  '000660
2    LG에너지솔루션  '373220
3    삼성바이오로직스  '207940
4         현대차  '005380
5          기아  '000270
6        셀트리온  '068270
7        KB금융  '105560
8       NAVER  '035420
9        신한지주  '055550
10      현대모비스  '012330
11   POSCO홀딩스  '005490
12       고려아연  '010130
13       삼성물산  '028260
14       LG화학  '051910
15       삼성생명  '032830
16    메리츠금융지주  '138040
17    HD현대중공업  '329180
18  한화에어로스페이스  '012450
19     하나금융지주  '086790
20      삼성SDI  '006400
21        HMM  '011200
22       삼성화재  '000810
23        카카오  '035720
24       KT&G  '033780
25       한국전력  '015760
26       LG전자  '066570
27    두산에너빌리티  '034020
28       크래프톤  '259960
29   HD한국조선해양  '009540


In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import urllib.parse
import os

def collect_news_for_date(search_query, code, date):
    news_list = []
    encoded_query = urllib.parse.quote(search_query.encode('euc-kr'))
    url = f"https://finance.naver.com/news/news_search.naver?q={encoded_query}&sm=all.basic&pd=1&stDateStart={date}&stDateEnd={date}"
    response = requests.get(url)
    response.encoding = 'euc-kr'
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('dd', class_='articleSubject')[:1]  # Limit to 1 news title per date
    for article in articles:
        title = article.find('a').text.strip()
        news_list.append({'Date': date, 'Code': code, 'Company': search_query, 'Title': title})
    return news_list

def collect_and_save_all_news(df, start_date, end_date):
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')
    
    # 빈 데이터프레임 생성
    result_df = pd.DataFrame(columns=['Date', 'Code', 'Company', 'Title'])

    # 뉴스 데이터 수집
    for _, row in df.iterrows():
        company = row['Company']
        code = row['Code']
        current_date = start_date
        while current_date <= end_date:
            # 월, 화, 수, 목, 일(평일+일요일)만 수집
            if current_date.weekday() in [0, 1, 2, 3, 6]:  # 월: 0, 화: 1, 수: 2, 목: 3, 일: 6
                date_str = current_date.strftime('%Y-%m-%d')
                news = collect_news_for_date(company, code, date_str)
                if news:
                    # 수집된 뉴스 데이터 추가
                    result_df = pd.concat([result_df, pd.DataFrame(news)], ignore_index=True)
            
            # 다음 날짜로 이동
            current_date += timedelta(days=1)
    
    # 데이터 저장
    output_path = 'Data/news_data.csv'
    os.makedirs('Data', exist_ok=True)
    result_df.to_csv(output_path, index=False, encoding='utf-8-sig')

    print(f"모든 뉴스 수집이 완료되었습니다. 파일은 {output_path}에 저장되었습니다.")

# 주식 데이터 로드 및 뉴스 수집 실행
df = pd.read_csv('Data/kopsi_stocks.csv')
collect_and_save_all_news(df, '2021-01-01', '2023-12-31')

모든 뉴스 수집이 완료되었습니다. 파일은 Data/news_data.csv에 저장되었습니다.
